### Food Image Classification With a CNN w/ Transfer Learning via VGG16

In [1]:
# load the necessary packages
import tensorflow as tf
from tensorflow.keras.layers import Input, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import Model
import pandas as pd

import keras.backend as K


import os
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Load and Preprocess Data for Model

In [2]:
# setup model architechture
# remove dense top layers from pre-trained model
base_model = VGG16(weights='imagenet', 
              include_top=False, 
              input_tensor=(Input(shape=(224,224,3)))
             )

#freeze training in pre-trained layers
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
#add in final layers of model
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(4,4))(head_model)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.4)(head_model)
head_model = Dense(101, activation='softmax')(head_model)

#place new head model on top of pre-trained network
model = Model(inputs=base_model.input, outputs=head_model)

In [4]:
def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [5]:
csv_logger = CSVLogger('../history/VGG16_v4.log')

# initialize early stopping callback
es_callback = EarlyStopping(monitor="val_loss", patience=10)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
batch_size = 25
epochs = 1000
IMG_HEIGHT = 224
IMG_WIDTH = 224
train_dir = '../data/train'
valid_dir = '../data/valid'
dropout_rate = 0.4

In [8]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                          horizontal_flip=True,
                                          zoom_range=[0.5,1],
                                          channel_shift_range=40)
valid_image_generator = ImageDataGenerator(rescale=1./255)

In [9]:
# initialize training image iterator
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=train_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                          seed=42)

Found 75750 images belonging to 101 classes.


In [10]:
# initialize test image iterator
valid_data_gen = valid_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=valid_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                          seed=42)

Found 12625 images belonging to 101 classes.


In [11]:
# compile model
model.compile(optimizer='adam',
            loss=euclidean_distance_loss,
            metrics=['accuracy'])

In [12]:
# train model
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=75750//batch_size,
    epochs=epochs,
    validation_data=valid_data_gen,
    validation_steps=12625//batch_size,
    workers=1,
    use_multiprocessing=False,
    callbacks=[es_callback, csv_logger]
)

Epoch 1/1000
3030/3030 [==============================] - 1577s 521ms/step - loss: 0.9817 - accuracy: 0.0762 - val_loss: 0.9593 - val_accuracy: 0.1393
Epoch 2/1000
3030/3030 [==============================] - 1569s 518ms/step - loss: 0.9637 - accuracy: 0.1210 - val_loss: 0.9424 - val_accuracy: 0.1781
Epoch 3/1000
3030/3030 [==============================] - 1584s 523ms/step - loss: 0.9558 - accuracy: 0.1354 - val_loss: 0.9334 - val_accuracy: 0.1827
Epoch 4/1000
3030/3030 [==============================] - 1591s 525ms/step - loss: 0.9501 - accuracy: 0.1481 - val_loss: 0.9272 - val_accuracy: 0.1915
Epoch 5/1000
3030/3030 [==============================] - 1586s 523ms/step - loss: 0.9455 - accuracy: 0.1541 - val_loss: 0.9139 - val_accuracy: 0.2139
Epoch 6/1000
3030/3030 [==============================] - 1572s 519ms/step - loss: 0.9433 - accuracy: 0.1573 - val_loss: 0.9187 - val_accuracy: 0.2078
Epoch 7/1000
3030/3030 [==============================] - 1565s 517ms/step - loss: 0.9405 - ac

KeyboardInterrupt: 

In [ ]:
# if there is not a saved model, save model for future use
if not os.path.isdir('../saved_models'):
    os.makedirs('../saved_models')
model.save('../saved_models/CNN_VGG16_MODEL_V4.h5')

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()